# ImageClassification with ResNet

Source: "https://learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/"

Importo i modelli disponibili in TorchVision e li mostro a display

In [1]:
from torchvision import models, transforms
from PIL import Image
import torch

# dir(models)

Scelta la ResNet, viene impostata già pre-allenata (sis caricano tutti i pesi associati ai vari layer).
In particolare la ResNet152 è stata scelta, presentando appunto 152 layer, suppongo possa avere un rendimento migliore.

In [2]:
ResNet = models.resnet152(pretrained=True)
#print(ResNet)

C:\Users\antoa\anaconda3\envs\EsercLu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\antoa\anaconda3\envs\EsercLu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Pre-impostazione della trasformazione richiesta alla generica immagine che verrà caricata poi in seguito, adattandola all'input richiesto dalla ResNet.
Nell'esempio portava anche una funzione di cropping a 224x224 a valle di un resize di 256x256: pensavo fosse superfluo e poteva portare a perdite di informazioni nell'immagine (es. un oggetto non perfettamente centrato). I valori di normalizzazione sono preimpostati seguendo i valori esposti nel link di Luca.

In [3]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std  = [0.229, 0.224, 0.225]
    )
])

Carico l'imagine di Renato, che dovrebbe appartenere alla classe di riga 30 della ResNet

In [4]:
img = Image.open("C:/Users/antoa/ImageClassification/img/Lucio.jpg")
img.show()

Conversione dell'immagine nell'input richiesto da ResNet;

Cos'è il batch?

In [5]:
img_t = transform(img)
batch_t = torch.unsqueeze(img_t,0)

Inizializzo il modello ponendolo in formato eval ???
Identifico la classe prevista

In [6]:
ResNet.eval()
output = ResNet(batch_t)
#print(output.shape)

Dal file di testo in cui ho tutte le classi che posso identificare, creo il vettore stringa in cui ogni classe è associata ad un indice

In [7]:
with open('C:/Users/antoa/ImageClassification/imagenet_classes.txt') as f:
          classes = [line.strip() for line in f.readlines()]
#print("Number of classes: {}".format(len(classes)))
#classes

Prelevo gli indici del vettore output ordinato in senso decrescente

Valuto il valore percentuale di ognuno con softmax

Prelevo i primi 10 valori maggiori

In [8]:
#_, index = torch.max(output, 1)
#percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
#print(classes[index[0]], percentage[index[0]].item())

_, index = torch.sort(output, descending=True)
percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
[(classes[idx], percentage[idx].item()) for idx in index[0][:10]]

[('chimpanzee, chimp, Pan troglodytes', 98.89942169189453),
 ('macaque', 0.4933602213859558),
 ('siamang, Hylobates syndactylus, Symphalangus syndactylus',
  0.12610672414302826),
 ('guenon, guenon monkey', 0.0650237575173378),
 ('orangutan, orang, orangutang, Pongo pygmaeus', 0.051429834216833115),
 ('spider monkey, Ateles geoffroyi', 0.04538612440228462),
 ('gorilla, Gorilla gorilla', 0.03555387258529663),
 ('baboon', 0.030215026810765266),
 ('patas, hussar monkey, Erythrocebus patas', 0.029908213764429092),
 ('sloth bear, Melursus ursinus, Ursus ursinus', 0.02793831191956997)]